In [ ]:
import pandas as pd
import random
import numpy as np

In [ ]:
def to_sequence(data, seq_size=10):
    x, y = [], []

    for i in range(len(data)-seq_size):
        window = data[i:(i+seq_size)].values
        after_window = data[i+seq_size]
        x.append(window)
        y.append(after_window)

    return np.array(x), np.array(y)

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import *

vertical_MAE =[]
vertical_MSE=[]
vertical_RMSE =[]
vertical_MRE=[]
vertical_MMRE =[]
vertical_PRED=[]

for j in range(1,103):
  # Read csv file into a pandas dataframe
  df = pd.read_csv(f'/content/Patient_{j}.csv')
  print(j)

  for i in range(len(df)):
    if np.isnan(df.at[i,'SensorGLU']):
      X, y = to_sequence(df[:i-1].SensorGLU)
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1)
      # Create a SVR model
      model = SVR(kernel='rbf',gamma=0.5, C=10, epsilon = 0.05)
      # Train the model
      model.fit(X_train, y_train)
      # Make Predictions and Assess Model
      pred = model.predict(X_test)
      df.at[i,'SensorGLU']=pred



  vertical_MAE =np.append(vertical_MAE , mean_absolute_error(y_test,pred))
  vertical_MSE =np.append(vertical_MSE , mean_squared_error(y_test,pred))
  vertical_RMSE =np.append(vertical_RMSE , np.sqrt(mean_squared_error(y_test,pred)))
  vertical_MRE = np.append(vertical_MRE , np.abs((y_test-pred)/y_test)*100)
  vertical_MMRE = np.append(vertical_MMRE , np.mean(np.abs((y_test-pred)/y_test)*100))
  Pred = lambda p: len(vertical_MRE[vertical_MRE <= p])/len(vertical_MRE)*100
  PRED = Pred(25)
  vertical_PRED =np.append(vertical_PRED , PRED)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102


In [ ]:
print(vertical_MAE)

[1.98491721e+01 2.46527335e+01 7.01571763e+01 1.87934082e+02
 3.83017727e+01 3.71081815e+01 2.30862538e+01 1.15369960e+02
 4.96068529e-02 1.23820311e+01 1.05766640e+02 7.84348895e+01
 4.31027067e+01 3.19226623e+01 3.41074929e+00 6.91910756e+01
 4.41781994e+01 5.77739743e+01 1.33064705e+02 1.02845778e+02
 3.11074228e+01 2.53131051e+01 1.78851021e+02 1.62042944e+02
 1.92374132e+00 9.29982818e+00 7.54752065e+00 1.62076504e+01
 1.13189659e+02 4.24208737e+01 3.27746856e+01 4.62861141e+00
 5.18784600e+00 4.72547662e+00 5.40678459e+01 6.66911480e+01
 9.15357143e+01 4.15054620e+01 3.83877734e+00 2.25506503e-01
 1.01906570e+01 2.35566866e+02 3.73455660e+01 7.42451057e+01
 5.61477101e+01 5.47510221e+01 1.03329952e+02 2.00333684e+02
 4.81892825e+01 1.12292489e+02 2.35233429e+00 1.50679328e+01
 2.90002314e+00 8.85755259e+01 4.76136096e+01 4.10504961e+01
 1.61351540e+01 3.18985763e+01 5.66484187e+00 2.24009711e+01
 1.01615361e+02 4.36449060e+01 3.60608934e+01 1.58664162e+01
 9.42450119e+01 8.767611

In [ ]:
len(vertical_MAE)

102

In [ ]:
import csv


test = pd.DataFrame({"MAE": vertical_MAE, "RMSE": vertical_RMSE, "MMRE": vertical_MMRE, "PRED": vertical_PRED})
test.to_csv(r'/content/Patient_result_SVR_5.csv', index=False)

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from keras.layers import RNN, SimpleRNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import *

vertical_MAE =[]
vertical_MSE=[]
vertical_RMSE =[]
vertical_MRE=[]
vertical_MMRE =[]
vertical_PRED=[]

for j in range(1,40):
  # Read csv file into a pandas dataframe
  df = pd.read_csv(f'/content/Patient_{j}.csv')
  print(j)

  for i in range(len(df)):
    if np.isnan(df.at[i,'SensorGLU']):
      X, y = to_sequence(df[:i-1].SensorGLU)
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1)
      X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
      X_test = np.reshape(X_test, (X_test.shape[0],  X_test.shape[1],1))
      # Define the RNN model
      model = Sequential()
      model.add(SimpleRNN(64))
      model.add(Dense(1))
      # Compile the model
      model.compile(loss='mean_squared_error', optimizer='adam')
      # Train the model
      model.fit(X_train, y_train, epochs=50,batch_size = 32, verbose=0)
      # Make Predictions
      pred = model.predict(X_test)
      df.at[i,'SensorGLU']=pred


  vertical_MAE =np.append(vertical_MAE , mean_absolute_error(y_test,pred))
  vertical_MSE =np.append(vertical_MSE , mean_squared_error(y_test,pred))
  vertical_RMSE =np.append(vertical_RMSE , np.sqrt(mean_squared_error(y_test,pred)))
  vertical_MRE = np.append(vertical_MRE , np.abs((y_test-pred)/y_test)*100)
  vertical_MMRE = np.append(vertical_MMRE , np.mean(np.abs((y_test-pred)/y_test)*100))
  Pred = lambda p: len(vertical_MRE[vertical_MRE <= p])/len(vertical_MRE)*100
  PRED = Pred(25)
  vertical_PRED =np.append(vertical_PRED , PRED)
  tf.keras.backend.clear_session()  # Clear TensorFlow session to free up memory
  print("Tensorflow/keras session cleared")


1
1/1 [==============================] - 0s 242ms/step


1/1 [==============================] - 0s 143ms/step


1/1 [==============================] - 0s 139ms/step
Tensorflow/keras session cleared
2
1/1 [==============================] - 0s 140ms/step
Tensorflow/keras session cleared
3
1/1 [==============================] - 0s 168ms/step
Tensorflow/keras session cleared
4
1/1 [==============================] - 0s 215ms/step
Tensorflow/keras session cleared
5
1/1 [==============================] - 0s 158ms/step
Tensorflow/keras session cleared
6
1/1 [==============================] - 0s 149ms/step
Tensorflow/keras session cleared
7
1/1 [==============================] - 0s 205ms/step
Tensorflow/keras session cleared
8
1/1 [==============================] - 0s 137ms/step
Tensorflow/keras session cleared
9
1/1 [==============================] - 0s 193ms/step
Tensorflow/keras session cleared
10
1/1 [==============================] - 0s 137ms/step
Tensorflow/keras session cleared
11
1/1 [==============================] - 0s 140ms/step
Tensorflow/keras session cleared
12
1/1 [========================

In [ ]:
import csv


test = pd.DataFrame({"MAE": vertical_MAE, "RMSE": vertical_RMSE, "MMRE": vertical_MMRE, "PRED": vertical_PRED})
test.to_csv(r'/content/Patient_result_RNN_5_part1.csv', index=False)